# Predicting Jay Chou Data 數據預測周杰倫

## by Brian Cheng

This is a Python Jupyter notebook where I use Spotify API data and data I compiled myself to predict what a Jay Chou new song's Spotify Popularity will be.

### 1. Import packages

In [1]:
#!pip3 install spotipy
import spotipy #Spotify API
from spotipy.oauth2 import SpotifyClientCredentials #Spotify client credentials
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

### 2. Compiling the data

#### 2.1 Accessing the data from Spotify API

##### 2.1.1 Accessing all the studio album tracks from Spotify

In [2]:
#Spotify API authentication token
#CHANGE IT TO YOUR OWN if you're running my code
CLIENT_ID = 'INPUT_YOUR_OWN'
CLIENT_SECRET = 'INPUT_YOUR_OWN'
#access the Spotify API
sp = spotipy.Spotify(client_credentials_manager = SpotifyClientCredentials(
    client_id = CLIENT_ID, client_secret = CLIENT_SECRET)
                    )

In [3]:
#access the playlist I made containing all Jay Chou's studio album tracks (so far, up to 2016)
playlist = sp.user_playlist_tracks('b5jfei3kyhlyuiadni4zvzl84', 'spotify:playlist:2rivA3vIiO51ooKBGWv7hh')

In [4]:
#get all the tracks from the playlist (while loop because Spotify only allows getting 100 tracks at a time)
tracks = playlist['items']
while playlist['next']:
    playlist = sp.next(playlist)
    tracks.extend(playlist['items'])
tracks_df = pd.DataFrame()

##### 2.1.2 Obtaining the basic info for every track

In [5]:
#obtain basic info for the tracks
for track in tracks:
    track_df = pd.DataFrame(pd.DataFrame(track)['track']).transpose()
    tracks_df = tracks_df.append(track_df)
tracks_df = tracks_df.reset_index(drop = True)
tracks_df.head()

,external_urls,href,id,type,uri,album,artists,available_markets,disc_number,duration_ms,episode,explicit,external_ids,is_local,name,popularity,preview_url,track,track_number,url
0,{'spotify': 'https://open.spotify.com/track/4R...,https://api.spotify.com/v1/tracks/4RbjlqzGc5pi...,4RbjlqzGc5piUTTr32XMlv,track,spotify:track:4RbjlqzGc5piUTTr32XMlv,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,236466,False,False,{'isrc': 'TWK970000101'},False,可愛女人,50,https://p.scdn.co/mp3-preview/5884de04bd6095d7...,True,1,NaN
1,{'spotify': 'https://open.spotify.com/track/66...,https://api.spotify.com/v1/tracks/669LQQmGgcpQ...,669LQQmGgcpQqfblioj5qR,track,spotify:track:669LQQmGgcpQqfblioj5qR,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,241973,False,False,{'isrc': 'TWK970000102'},False,完美主義,33,https://p.scdn.co/mp3-preview/afe7c9a5821edc07...,True,2,NaN
2,{'spotify': 'https://open.spotify.com/track/4V...,https://api.spotify.com/v1/tracks/4VWn7L2kONeM...,4VWn7L2kONeMEQ6cAIfzXC,track,spotify:track:4VWn7L2kONeMEQ6cAIfzXC,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,256666,False,False,{'isrc': 'TWK970000103'},False,星晴,52,https://p.scdn.co/mp3-preview/1c6c55493a6363c7...,True,3,NaN
3,{'spotify': 'https://open.spotify.com/track/0E...,https://api.spotify.com/v1/tracks/0EEDSjk382WM...,0EEDSjk382WMP9PKsYPKIZ,track,spotify:track:0EEDSjk382WMP9PKsYPKIZ,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,268466,False,False,{'isrc': 'TWK970000104'},False,娘子,33,https://p.scdn.co/mp3-preview/c6eed0939bea6e44...,True,4,NaN
4,{'spotify': 'https://open.spotify.com/track/0M...,https://api.spotify.com/v1/tracks/0MXi5biqp5KJ...,0MXi5biqp5KJw2wHh89o03,track,spotify:track:0MXi5biqp5KJw2wHh89o03,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,276333,False,False,{'isrc': 'TWK970000105'},False,鬥牛,32,https://p.scdn.co/mp3-preview/1529040c222e7d41...,True,5,NaN


##### 2.1.3 Editing the basic info for every track

In [6]:
#extract release date as a column
tracks_df['release_date'] = tracks_df.apply(lambda x: x['album']['release_date'], axis = 1)
#extract album name as a column
tracks_df['album'] = tracks_df.apply(lambda x: x['album']['name'], axis = 1)

In [7]:
#correct the release dates (some release date data on Spotify are incorrect)
def correctReleaseDates(x):
    if x == '2002-07-30':
        return '2002-07-18'
    if x == '2003-11-01':
        return '2003-11-12'
    if x == '2006-01-16':
        return '2006-01-20'
    if x == '2007-08-10':
        return '2007-08-13'
    if x == '2008-01-29':
        return '2008-01-30'
    if x == '2008-03-10':
        return '2007-11-02'
    if x == '2008-11-09':
        return '2008-10-14'
    if x == '2010-05-20':
        return '2010-05-18'
    if x == '2011-11-16':
        return '2011-11-11'
    else:
        return x
tracks_df['release_date'] = tracks_df['release_date'].apply(correctReleaseDates)

In [8]:
#change available_markets to number of markets
tracks_df['available_markets'] = tracks_df['available_markets'].apply(lambda x: len(x))

In [9]:
#change release_date to days since
tracks_df['release_date'] = tracks_df['release_date'].apply(lambda x: (datetime.date.today() - datetime.datetime.strptime(str(x), '%Y-%m-%d').date()).days)


In [10]:
#correct the names (some names were inputted wrong on Spotify)
def correctNames(x):
    if x == '蛇 舞':
        return '蛇舞'
    if x == '花 海':
        return '花海'
    if x == '爸我回來了':
        return '爸 我回來了'
    if x == '我落淚˙情緒零碎':
        return '我落淚 情緒零碎'
    else:
        return x
tracks_df['name'] = tracks_df['name'].apply(correctNames)

In [11]:
#drop unnecessary features from our basic info
tracks_df = tracks_df[['name', 'id', 'track_number', 'popularity', 'release_date']]
tracks_df['track_number'] = pd.to_numeric(tracks_df['track_number'])
tracks_df['popularity'] = pd.to_numeric(tracks_df['popularity'])
tracks_df.head()

,name,id,track_number,popularity,release_date
0,可愛女人,4RbjlqzGc5piUTTr32XMlv,1,50,7473
1,完美主義,669LQQmGgcpQqfblioj5qR,2,33,7473
2,星晴,4VWn7L2kONeMEQ6cAIfzXC,3,52,7473
3,娘子,0EEDSjk382WMP9PKsYPKIZ,4,33,7473
4,鬥牛,0MXi5biqp5KJw2wHh89o03,5,32,7473


##### 2.1.4 Obtaining the audio features for every track

In [12]:
#audio features for the tracks
audio_features_df = pd.DataFrame()
for index, row in tracks_df.iterrows():
    audio_feature = pd.DataFrame(sp.audio_features('spotify:track:' + row['id']))
    audio_features_df = audio_features_df.append(audio_feature)
audio_features_df = audio_features_df.reset_index(drop = True)
audio_features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.767,0.671,5,-9.995,1,0.0317,0.552,0.000032,0.201,0.921,135.468,audio_features,4RbjlqzGc5piUTTr32XMlv,spotify:track:4RbjlqzGc5piUTTr32XMlv,https://api.spotify.com/v1/tracks/4RbjlqzGc5pi...,https://api.spotify.com/v1/audio-analysis/4Rbj...,236467,4
1,0.658,0.605,2,-9.036,0,0.1740,0.627,0.000000,0.346,0.643,89.889,audio_features,669LQQmGgcpQqfblioj5qR,spotify:track:669LQQmGgcpQqfblioj5qR,https://api.spotify.com/v1/tracks/669LQQmGgcpQ...,https://api.spotify.com/v1/audio-analysis/669L...,241973,4
2,0.637,0.329,7,-10.366,1,0.0349,0.725,0.000009,0.245,0.255,177.792,audio_features,4VWn7L2kONeMEQ6cAIfzXC,spotify:track:4VWn7L2kONeMEQ6cAIfzXC,https://api.spotify.com/v1/tracks/4VWn7L2kONeM...,https://api.spotify.com/v1/audio-analysis/4VWn...,256667,4
3,0.874,0.620,5,-8.700,0,0.0555,0.306,0.000000,0.117,0.832,136.000,audio_features,0EEDSjk382WMP9PKsYPKIZ,spotify:track:0EEDSjk382WMP9PKsYPKIZ,https://api.spotify.com/v1/tracks/0EEDSjk382WM...,https://api.spotify.com/v1/audio-analysis/0EED...,268467,4
4,0.844,0.645,6,-9.175,0,0.0770,0.437,0.000027,0.371,0.735,129.895,audio_features,0MXi5biqp5KJw2wHh89o03,spotify:track:0MXi5biqp5KJw2wHh89o03,https://api.spotify.com/v1/tracks/0MXi5biqp5KJ...,https://api.spotify.com/v1/audio-analysis/0MXi...,276333,4


##### 2.1.5 Editing the audio features for every track

In [13]:
#drop unnecessary audio features
audio_features_df = audio_features_df.drop(columns = ['analysis_url', 'liveness', 'track_href', 'type', 'uri'])
audio_features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,valence,tempo,id,duration_ms,time_signature
0,0.767,0.671,5,-9.995,1,0.0317,0.552,0.000032,0.921,135.468,4RbjlqzGc5piUTTr32XMlv,236467,4
1,0.658,0.605,2,-9.036,0,0.1740,0.627,0.000000,0.643,89.889,669LQQmGgcpQqfblioj5qR,241973,4
2,0.637,0.329,7,-10.366,1,0.0349,0.725,0.000009,0.255,177.792,4VWn7L2kONeMEQ6cAIfzXC,256667,4
3,0.874,0.620,5,-8.700,0,0.0555,0.306,0.000000,0.832,136.000,0EEDSjk382WMP9PKsYPKIZ,268467,4
4,0.844,0.645,6,-9.175,0,0.0770,0.437,0.000027,0.735,129.895,0MXi5biqp5KJw2wHh89o03,276333,4


#### 2.2 Import the music credits data

In [14]:
#read in the credits data I compiled
credits_df = pd.read_csv('Jay Chou - Main Albums.csv').drop(columns = ['year', 'album', 'arranger'])
credits_df

,name,lyricist
0,可愛女人,徐若瑄
1,完美主義,方文山
2,星晴,周杰倫
3,娘子,方文山
4,鬥牛,方文山
...,...,...
145,不該,方文山
146,土耳其冰淇淋,周杰倫
147,告白氣球,方文山
148,Now You See Me,方文山


#### 2.3 Combine the Spotify API data with my compiled data

In [15]:
#combine basic data of tracks and their audio features together
df = credits_df.merge(tracks_df, on = 'name')
df = df.merge(audio_features_df, on = 'id').drop(columns = 'id')
df

,name,lyricist,track_number,popularity,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,valence,tempo,duration_ms,time_signature
0,可愛女人,徐若瑄,1,50,7473,0.767,0.671,5,-9.995,1,0.0317,0.55200,0.000032,0.921,135.468,236467,4
1,完美主義,方文山,2,33,7473,0.658,0.605,2,-9.036,0,0.1740,0.62700,0.000000,0.643,89.889,241973,4
2,星晴,周杰倫,3,52,7473,0.637,0.329,7,-10.366,1,0.0349,0.72500,0.000009,0.255,177.792,256667,4
3,娘子,方文山,4,33,7473,0.874,0.620,5,-8.700,0,0.0555,0.30600,0.000000,0.832,136.000,268467,4
4,鬥牛,方文山,5,32,7473,0.844,0.645,6,-9.175,0,0.0770,0.43700,0.000027,0.735,129.895,276333,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,不該,方文山,6,57,1764,0.441,0.619,4,-6.609,1,0.0293,0.19000,0.000000,0.187,75.027,290133,4
146,土耳其冰淇淋,周杰倫,7,31,1764,0.662,0.924,7,-6.473,1,0.0417,0.00203,0.206000,0.602,114.015,195760,4
147,告白氣球,方文山,8,63,1764,0.590,0.572,11,-7.658,1,0.0379,0.19200,0.000000,0.448,179.989,215147,4
148,Now You See Me,方文山,9,34,1764,0.566,0.851,5,-5.549,0,0.0284,0.00202,0.000000,0.552,93.009,174053,4


In [16]:
#one-hot encode lyricists
one_hot_lyricist = pd.get_dummies(df['lyricist'])
df = df.drop('lyricist', axis = 1)
df = df.join(one_hot_lyricist)
df  

,name,track_number,popularity,release_date,danceability,energy,key,loudness,mode,speechiness,...,徐若瑄,從從,方文山,曾郁婷,林義傑,"羅宇軒, 李汪哲","羅宇軒, 黃婕熙",許世昌,黃俊郎,黃凌嘉
0,可愛女人,1,50,7473,0.767,0.671,5,-9.995,1,0.0317,...,1,0,0,0,0,0,0,0,0,0
1,完美主義,2,33,7473,0.658,0.605,2,-9.036,0,0.1740,...,0,0,1,0,0,0,0,0,0,0
2,星晴,3,52,7473,0.637,0.329,7,-10.366,1,0.0349,...,0,0,0,0,0,0,0,0,0,0
3,娘子,4,33,7473,0.874,0.620,5,-8.700,0,0.0555,...,0,0,1,0,0,0,0,0,0,0
4,鬥牛,5,32,7473,0.844,0.645,6,-9.175,0,0.0770,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,不該,6,57,1764,0.441,0.619,4,-6.609,1,0.0293,...,0,0,1,0,0,0,0,0,0,0
146,土耳其冰淇淋,7,31,1764,0.662,0.924,7,-6.473,1,0.0417,...,0,0,0,0,0,0,0,0,0,0
147,告白氣球,8,63,1764,0.590,0.572,11,-7.658,1,0.0379,...,0,0,1,0,0,0,0,0,0,0
148,Now You See Me,9,34,1764,0.566,0.851,5,-5.549,0,0.0284,...,0,0,1,0,0,0,0,0,0,0


### 3. Testing out the model

Test and train a model on available data first to see if what I'm doing is doable.

In [17]:
#split 60% of data as training and 40% of data as testing
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns = ['name', 'popularity']), df['popularity'], test_size = 0.3)


In [18]:
#random forest
rfr = RandomForestRegressor()
rfr.fit(StandardScaler().fit_transform(X_train), y_train)

RandomForestRegressor()

In [19]:
#gradient boosting
gbr = GradientBoostingRegressor()
gbr.fit(StandardScaler().fit_transform(X_train), y_train)

GradientBoostingRegressor()

In [20]:
#predictions from random forest
predictions1 = rfr.predict(StandardScaler().fit_transform(X_test))

In [21]:
#predictions from gradient boosting
predictions2 = gbr.predict(StandardScaler().fit_transform(X_test))

In [22]:
#r-squared for random forest
score1 = rfr.score(StandardScaler().fit_transform(X_test), y_test)
score1

0.4110572845868248

In [23]:
#r-squared for gradient boosting
score2 = gbr.score(StandardScaler().fit_transform(X_test), y_test)
score2

0.30231981313076206

In [24]:
#average the predictions
predictions = (predictions1 + predictions2) / 2

In [25]:
#how off the model is on average
mean_squared_error(y_test, predictions) ** 0.5

7.817591704284327

### 4. Prepare the new song we want to predict

#### 4.1 Get the song

In [26]:
#Spotify URI for the song we are predicting for
songURL = 'spotify:track:7BYMjh3vWnHU4IFyfZdIDO'

#### 4.2 Get the track info

In [27]:
#get track info of the song
track_df = pd.Series(sp.track(songURL)).to_frame().T
track_df

,album,artists,available_markets,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,name,popularity,preview_url,track_number,type,uri
0,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,185008,False,{'isrc': 'TWK972001101'},{'spotify': 'https://open.spotify.com/track/7B...,https://api.spotify.com/v1/tracks/7BYMjh3vWnHU...,7BYMjh3vWnHU4IFyfZdIDO,False,Mojito,58,https://p.scdn.co/mp3-preview/9d68ec32f79cd83e...,1,track,spotify:track:7BYMjh3vWnHU4IFyfZdIDO


#### 4.3 Edit the track info

In [28]:
#extract release date as a column
track_df['release_date'] = track_df.apply(lambda x: x['album']['release_date'], axis = 1)
#change release_date to days since
track_df['release_date'] = track_df['release_date'].apply(lambda x: (datetime.date.today() - datetime.datetime.strptime(str(x), '%Y-%m-%d').date()).days)
track_df = track_df[['name', 'id', 'track_number', 'popularity', 'release_date']]
track_df

,name,id,track_number,popularity,release_date
0,Mojito,7BYMjh3vWnHU4IFyfZdIDO,1,58,315


#### 4.4 Get the audio features

In [29]:
#get the audio features of the song
audio_features_track_df = pd.DataFrame(sp.audio_features(songURL)).drop(columns = ['analysis_url', 'liveness', 'track_href', 'type', 'uri'])
audio_features_track_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,valence,tempo,id,duration_ms,time_signature
0,0.814,0.595,0,-7.726,1,0.0277,0.476,0,0.8,115.004,7BYMjh3vWnHU4IFyfZdIDO,185008,4


#### 4.5 Merge the track info and audio features

In [30]:
#merge track info and audio features
track_df = track_df.merge(audio_features_track_df, on = 'id').drop(columns = 'id')
track_df

,name,track_number,popularity,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,valence,tempo,duration_ms,time_signature
0,Mojito,1,58,315,0.814,0.595,0,-7.726,1,0.0277,0.476,0,0.8,115.004,185008,4


#### 4.6 Manually add the lyricist

In [31]:
#manually input the lyricist for the new song to be predicted
TRACK_LYRICIST = '黃俊郎'

In [32]:
#add lyricist data to the track's data
for column in df.columns:
    if column not in track_df.columns:
        track_df[column] = 0
track_df[TRACK_LYRICIST] = 1
track_df

,name,track_number,popularity,release_date,danceability,energy,key,loudness,mode,speechiness,...,徐若瑄,從從,方文山,曾郁婷,林義傑,"羅宇軒, 李汪哲","羅宇軒, 黃婕熙",許世昌,黃俊郎,黃凌嘉
0,Mojito,1,58,315,0.814,0.595,0,-7.726,1,0.0277,...,0,0,0,0,0,0,0,0,1,0


### 5. Predicting the new song's popularity

I am using all the data available to build the model this time and using boosting with 2 random forest regressors and 2 gradient boosting regressors.

In [33]:
#random forest 1
rfr1 = RandomForestRegressor()
rfr1.fit(StandardScaler().fit_transform(df.drop(columns = ['name', 'popularity'])), df['popularity'])

RandomForestRegressor()

In [34]:
#random forest 2
rfr2 = RandomForestRegressor()
rfr2.fit(StandardScaler().fit_transform(df.drop(columns = ['name', 'popularity'])), df['popularity'])

RandomForestRegressor()

In [35]:
#gradient boosting 1
gbr1 = GradientBoostingRegressor()
gbr1.fit(StandardScaler().fit_transform(df.drop(columns = ['name', 'popularity'])), df['popularity'])

GradientBoostingRegressor()

In [36]:
#gradient boosting 2
gbr2 = GradientBoostingRegressor()
gbr2.fit(StandardScaler().fit_transform(df.drop(columns = ['name', 'popularity'])), df['popularity'])

GradientBoostingRegressor()

In [37]:
#predictions from random forest 1
predictions1 = rfr1.predict(StandardScaler().fit_transform(track_df.drop(columns = ['name', 'popularity'])))

In [38]:
#predictions from random forest 2
predictions2 = rfr2.predict(StandardScaler().fit_transform(track_df.drop(columns = ['name', 'popularity'])))

In [39]:
#predictions from gradient boosting 1
predictions3 = gbr1.predict(StandardScaler().fit_transform(track_df.drop(columns = ['name', 'popularity'])))

In [40]:
#predictions from gradient boosting 2
predictions4 = gbr2.predict(StandardScaler().fit_transform(track_df.drop(columns = ['name', 'popularity'])))

In [41]:
#average the predictions
PREDICTION_FINAL = ((predictions1 + predictions2 + predictions3 + predictions4) / 4)[0]
PREDICTION_FINAL

42.21946230104162

In [42]:
#check what songs are similar to this predicted value
df[df['popularity'].between(PREDICTION_FINAL - 2, PREDICTION_FINAL + 2)]

,name,track_number,popularity,release_date,danceability,energy,key,loudness,mode,speechiness,...,徐若瑄,從從,方文山,曾郁婷,林義傑,"羅宇軒, 李汪哲","羅宇軒, 黃婕熙",許世昌,黃俊郎,黃凌嘉
6,伊斯坦堡,7,41,7473,0.633,0.479,2,-10.148,1,0.0428,...,1,0,0,0,0,0,0,0,0,0
11,爸 我回來了,2,44,7161,0.676,0.639,3,-7.008,0,0.1020,...,0,0,0,0,0,0,0,0,0,0
17,威廉古堡,8,42,7161,0.634,0.554,11,-6.572,0,0.0285,...,0,0,1,0,0,0,0,0,0,0
18,雙截棍,9,44,7161,0.701,0.876,7,-6.499,1,0.0643,...,0,0,1,0,0,0,0,0,0,0
25,分裂,6,44,6854,0.637,0.438,1,-8.504,1,0.0282,...,0,0,0,0,0,0,0,0,0,0
29,最後的戰役,10,44,6854,0.607,0.785,1,-3.810,1,0.0316,...,0,0,1,0,0,0,0,0,0,0
33,三年二班,4,42,6476,0.624,0.688,5,-8.857,0,0.0668,...,0,0,1,0,0,0,0,0,0,0
55,四面楚歌,5,44,5652,0.741,0.843,0,-3.260,0,0.0580,...,0,0,0,0,0,0,0,0,0,0
59,麥芽糖,9,43,5652,0.617,0.815,9,-4.553,1,0.0240,...,0,0,1,0,0,0,0,0,0,0
91,時光機,9,43,4574,0.450,0.626,0,-7.435,1,0.0364,...,0,0,1,0,0,0,0,0,0,0
